<a href="https://colab.research.google.com/github/GitAmitesh/SkinDiseaseClassificationCNN/blob/main/model_3_mobileNet_87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from scipy import stats

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


import tensorflow as tf
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from tensorflow.keras.regularizers import l2


In [2]:
!fusermount -u drive
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/drive


In [3]:
# Path to your .pkl file
file_path = "/content/drive/MyDrive/HAM10000/final_processed_ham_metadata.pkl"

# Load the DataFrame
df = pd.read_pickle(file_path)

# Check the result
df.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path,lesion_type,lesion_type_idx,image_array
0,HAM_0006018,ISIC_0033486,bkl,histo,75.0,male,trunk,/content/drive/MyDrive/HAM10000/HAM10000_image...,Benign keratosis-like lesions,2,"[[[0.62352943, 0.58431375, 0.61960787], [0.619..."
1,HAM_0000206,ISIC_0033925,mel,histo,65.0,female,upper extremity,/content/drive/MyDrive/HAM10000/HAM10000_image...,Melanoma,5,"[[[0.7529412, 0.57254905, 0.654902], [0.745098..."
2,HAM_0000673,ISIC_0029659,akiec,histo,70.0,female,face,/content/drive/MyDrive/HAM10000/HAM10000_image...,Actinic keratoses,0,"[[[0.75686276, 0.4862745, 0.4862745], [0.73333..."
3,HAM_0000000,aug_normal_skin_0298,ns,none,45.0,unknown,unknown,/content/drive/MyDrive/HAM10000/normal_skin_im...,Normal skin,7,"[[[0.30980393, 0.23529412, 0.1764706], [0.3058..."
4,HAM_0005505,ISIC_0031743,akiec,histo,50.0,male,upper extremity,/content/drive/MyDrive/HAM10000/HAM10000_image...,Actinic keratoses,0,"[[[0.015686275, 0.011764706, 0.003921569], [0...."


In [4]:
# Constants
SIZE = (150, 200, 3)
num_classes = 8
image_shape = (SIZE[0], SIZE[1], SIZE[2])
image_array_path = "/content/drive/MyDrive/HAM10000/final_processed_image_arrays.npy"  # This should be pre-saved memmap file path

# Load memory-mapped image array (not in RAM)
num_images = len(df)
image_arrays_loaded = np.load(image_array_path, mmap_mode='r')

# Labels
Y = df['lesion_type_idx']
Y_cat = to_categorical(Y, num_classes=num_classes)

# Train-test split (indices only)
train_idx, test_idx = train_test_split(np.arange(num_images), test_size=0.25, random_state=42)

In [5]:
# -----------------------------
# Custom Data Generator
# -----------------------------
class MemmapImageGenerator(Sequence):
    def __init__(self, memmap, labels, indices, batch_size=32):
        self.memmap = memmap
        self.labels = labels
        self.indices = indices
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_ids = self.indices[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_x = self.memmap[batch_ids]
        batch_y = self.labels[batch_ids]
        if batch_x is None or batch_y is None:
            raise ValueError("Generator is returning None values.")
        return batch_x, batch_y

# Create training and validation generators
batch_size = 64
train_gen = MemmapImageGenerator(image_arrays_loaded, Y_cat, train_idx, batch_size=batch_size)
test_gen = MemmapImageGenerator(image_arrays_loaded, Y_cat, test_idx, batch_size=batch_size)

In [7]:
# Define top-k accuracy metrics
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [12]:
# Load MobileNet without top layers
base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(150, 200, 3), pooling='avg')

# Freeze all layers except the last 23
for layer in base_model.layers[:-23]:
    layer.trainable = False

# Add custom head
x = base_model.output
x = Dropout(0.3)(x)
output = Dense(8, activation='softmax')(x)

<ipython-input-12-a57303ddb116>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(include_top=False, weights='imagenet', input_shape=(150, 200, 3), pooling='avg')


In [9]:
# Create model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy', top_2_accuracy, top_3_accuracy])

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

callbacks_list = [reduce_lr, early_stop]

# Train the model using your memory-efficient generators
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=test_gen,
    callbacks=callbacks_list,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
141/141 ━━━━━━━━━━━━━━━━━━━━ 101s 602ms/step - accuracy: 0.6347 - loss: 1.1678 - top_2_accuracy: 0.7981 - top_3_accuracy: 0.8781 - val_accuracy: 0.3959 - val_loss: 3.3745 - val_top_2_accuracy: 0.6670 - val_top_3_accuracy: 0.7969 - learning_rate: 0.0010
Epoch 2/15
141/141 ━━━━━━━━━━━━━━━━━━━━ 59s 415ms/step - accuracy: 0.8538 - loss: 0.3853 - top_2_accuracy: 0.9584 - top_3_accuracy: 0.9875 - val_accuracy: 0.5251 - val_loss: 2.4751 - val_top_2_accuracy: 0.6507 - val_top_3_accuracy: 0.7839 - learning_rate: 0.0010
Epoch 3/15
141/141 ━━━━━━━━━━━━━━━━━━━━ 61s 430ms/step - accuracy: 0.9161 - loss: 0.2399 - top_2_accuracy: 0.9809 - top_3_accuracy: 0.9958 - val_accuracy: 0.6813 - val_loss: 1.0991 - val_top_2_accuracy: 0.8715 - val_top_3_accuracy: 0.9560 - learning_rate: 0.0010
Epoch 4/15
141/141 ━━━━━━━━━━━━━━━━━━━━ 54s 386ms/step - accuracy: 0.9534 - loss: 0.1277 - top_2_accuracy: 0.9922 - top_3_accuracy: 0.9985 - val_accuracy: 0.7359 - val_loss: 0.9558 - val_top_2_accuracy: 0.8948 

In [10]:
# Evaluate
val_loss, val_acc, val_top_2_acc, val_top_3_acc = model.evaluate(test_gen)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)
print("Top-2 Accuracy:", val_top_2_acc)
print("Top-3 Accuracy:", val_top_3_acc)


47/47 ━━━━━━━━━━━━━━━━━━━━ 11s 232ms/step - accuracy: 0.8840 - loss: 0.4876 - top_2_accuracy: 0.9639 - top_3_accuracy: 0.9863
Validation Loss: 0.5039294958114624
Validation Accuracy: 0.8774558901786804
Top-2 Accuracy: 0.9623709917068481
Top-3 Accuracy: 0.9860140085220337


In [13]:
save_path = "/content/drive/MyDrive/mobilenet_best_model.keras"
model.save(save_path)